In [34]:
import netCDF4
from netCDF4 import Dataset
import os
from os.path import dirname
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib 
import matplotlib.pyplot as plt
from math import pi
from numpy import cos,sin
from scipy.spatial import cKDTree
from numpy import absolute as abs
import numpy.ma as ma
from ipywidgets import interact, interactive, fixed, interact_manual
from matplotlib import ticker
import ipywidgets as widgets


In [10]:
# Choose folder and resolution
#folder = dirname(dirname(os.getcwd())) + '\\Data_WQ\\' #The folder where the datasets are
folder = r"C:\Users\Rafael\Documents\Local\Mathematical Data Science\Project\Data_WQ\\"
resolution = '110m' # Change border resolution for the maps. Insert value in meters (e.g. '100m')

In [11]:
## Open dataset and store variables in var_values dictionary. Units are in var_units dictionary. Names in var_names
'''
Datasets:
    1: CHL-model
    2: CHL-satellite
    3: DOXYL-model
    4: NITR-model
    5: PHOS-model
    6: SPM-satellite
'''
# Define global variables

def select_dataset(num_dataset):
    # Define global variables
    global var_values, var_units, var_names, selected_variable, text, years_list, years_select, years
    
    # Open dataset
    if num_dataset == 2:
        file = folder + "dataset-CHL-satellite-daily.nc"
        selected_variable = 'CHL'
        text = 'Mass concentration of chlorophyll-a in sea water - satellite'
    elif num_dataset == 3:
        file = folder + "dataset-DOXYL-model-daily.nc"
        selected_variable = 'o2'
        text = 'Mole concentration of dissolved molecular oxygen in sea water - model'
    elif num_dataset == 4:
        file = folder + "dataset-NITR-model-daily.nc"
        selected_variable = 'no3'
        text = 'Mole concentration of nitrate in sea water - model'
    elif num_dataset == 5:
        file = folder + "dataset-PHOS-model-daily.nc"
        selected_variable = 'po4'
        text = 'Mole concentration of phosphate in sea water - model'
    elif num_dataset == 6:
        file = folder + "dataset-SPM-satellite-monthly.nc"
        selected_variable = 'SPM'
        text = 'Mass concentration of inorganic suspended matter in sea water - satellite'
    else:
        file = folder + "dataset-CHL-model-daily.nc"
        selected_variable = 'chl'
        text = 'Mass concentration of chlorophyll-a in sea water - model'

    dataset = Dataset(file, "r")


    var_values = {}
    var_units = {}
    var_names = {}

    for k in dataset.variables.keys():
        # Standardize keys 'latitude' and 'longitude'
        if k == 'lat':
            k_new = 'latitude'
        elif k == 'lon':
            k_new = 'longitude'
        else:
            k_new = k

        var_units[k_new] = dataset.variables[k].units # Save units
        var_names[k_new] = dataset.variables[k].long_name # Save name
        if 'time' in var_names[k_new]: #Save time
            var_values[k_new] = netCDF4.num2date(dataset.variables[k][:],var_units[k_new], only_use_cftime_datetimes=False, only_use_python_datetimes = True)
        else:
            var_values[k_new] = dataset.variables[k][:]

        # Remove single-dimensional entries
        var_values[k_new] = np.squeeze(var_values[k_new])

    # Close dataset
    dataset.close()

    years_list = list(map(lambda i : int(i.strftime("%Y")), var_values['time']))
    years_select = set(map(lambda i : int(i.strftime("%Y")), var_values['time']))
    years = set(map(lambda i : int(i.strftime("%Y")), var_values['time']))
    years_select.add('All')
    
    print('Dataset loaded. Using dataset {}'.format(file))

In [47]:
## List of years and indices

def plot_map_summary(sel, year, fixed_colorbar, min_colorbar, max_colorbar):
    
    if min_colorbar >= max_colorbar and fixed_colorbar == True:
        print('Error. The minimum limit for the colorbar must be smaller than the maximum limit of the colorbar')
        return
    
    text_plot = sel + " of " + text 
    
    if year == 'All':
        tmp = var_values[selected_variable]
    else:
        tmp = var_values[selected_variable][list(np.where(np.asarray(years_list) == year)[0]),:,:]
        text_plot += ' - ' + str(year)
    if sel == 'Max':
        var_plot = tmp.max(axis=0)
    elif sel == 'Min':
        var_plot = tmp.min(axis=0)
    elif sel == 'Mean':
        var_plot = tmp.mean(axis=0)
    elif sel == 'Deviation':
        var_plot = tmp.std(axis=0)
    elif sel == 'Argmax':
        var_plot = tmp.argmax(axis=0)
        var_plot = ma.masked_where(var_plot == 0, var_plot)
    elif sel == 'Argmin':
        var_plot = tmp.argmin(axis=0)
        var_plot = ma.masked_where(var_plot == 0, var_plot)
    else:
        var_plot = var_values[selected_variable].max(axis=0)     
        
    # Initialize plot
    matplotlib.rcParams['figure.figsize'] = (10,10) 

    # Initialize map
    proj=ccrs.Mercator()
    m = plt.axes(projection=proj)

    # Format map
    m.stock_img()
    m.coastlines(resolution=resolution)
    m.add_feature(cfeature.BORDERS)
    gl=m.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=2, color='gray', alpha=0.5, linestyle='--')
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabels_top = False
    gl.ylabels_right = False

    # Plot data

    if sel == 'Argmax' or sel == 'Argmin':
        plt.contourf(var_values['longitude'], var_values['latitude'], var_plot, 60, transform=ccrs.PlateCarree())
        # Add Colorbar
        cbar = plt.colorbar()
        if year == 'All':
            tick_locator = ticker.MaxNLocator(nbins=len(years))
            cbar.locator = tick_locator
            cbar.update_ticks()
            cbar.ax.set_yticklabels(years)
            cbar.set_label('Date')
        else:
            tick_locator = ticker.MaxNLocator(nbins=13)
            cbar.locator = tick_locator
            cbar.update_ticks()
            cbar.ax.set_yticklabels(list(np.floor(np.linspace(start=1, stop=tmp.shape[0], num=13)).astype(int)))
            cbar.set_label('Day of ' + str(year))
    else:
        if fixed_colorbar == True:
            levels = np.linspace(min_colorbar, max_colorbar, 60)
        else:
            levels = 60
        try:
            plt.contourf(var_values['longitude'], var_values['latitude'], var_plot, levels, transform=ccrs.PlateCarree())
        except ValueError:
            print('No values found for the selected colorbar limits. Please increase the range')
            return 
        # Add Colorbar
        cbar = plt.colorbar()
        cbar.set_label(var_units[selected_variable])

    # Add Title
    plt.title(text_plot)
    plt.show()

In [51]:
#Plot behaviuor thoughout years
#you can later select if you want to view min max mean or sd

def summary_years(sel):
    plt.figure(figsize=(16,4))
    text_plot = sel + " of " + text 
    tmp = var_values[selected_variable]
    years_list = list(map(lambda i : int(i.strftime("%Y")), var_values['time']))
    x = np.linspace(1998, 2018, 21)
    
        
    if sel == 'Max':
        maxi=[]
        for year in range(1998,2019,1):
            A=var_values[selected_variable][list(np.where(np.asarray(years_list) == year)),:,:]
            maxi=np.append(maxi,A.max())
        plt.plot(x,maxi)
        plt.xlabel('years')
               
    elif sel == 'Min':
        mini=[]
        for year in range(1998,2019,1):
            A=var_values[selected_variable][list(np.where(np.asarray(years_list) == year)),:,:]
            mini=np.append(mini,A.min())
        plt.plot(x,mini)
        plt.xlabel('years')
        
    elif sel == 'Mean':
        mean=[]
        for year in range(1998,2019,1):
            A=var_values[selected_variable][list(np.where(np.asarray(years_list) == year)),:,:]
            mean=np.append(mean,A.mean())
        plt.plot(x,mean)
        plt.xlabel('years')
        
    elif sel == 'Deviation':
        dv=[]
        for year in range(1998,2019,1):
            A=var_values[selected_variable][list(np.where(np.asarray(years_list) == year)),:,:]
            dv=np.append(dv,A.std())
        plt.plot(x,dv)
        plt.xlabel('years')
    

    # Add Title
    plt.title(text_plot)
    plt.xlabel('years')
    plt.xticks(x)

In [14]:
#plot behaviuours thoughout months
#you can later select if you want to view min max mean or sd

def summary_months(sel):
    plt.figure(figsize=(16,4))
    text_plot = sel + " of " + text 
    tmp = var_values[selected_variable]
    month_list=list(map(lambda i : int(i.strftime("%m")), var_values['time']))
    xm = np.linspace(1,12,12)
    
        
    if sel == 'Max':
        maxim=[]
        for month in range(1,13,1):
            Am=var_values[selected_variable][list(np.where(np.asarray(month_list) == month)),:,:]
            maxim=np.append(maxim,Am.max())
        plt.plot(xm,maxim)
        plt.xlabel('months')
        plt.title(text_plot)
               
    elif sel == 'Min':
        minim=[]
        for month in range(1,13,1):
            Am=var_values[selected_variable][list(np.where(np.asarray(month_list) == month)),:,:]
            minim=np.append(minim,Am.min())
        plt.plot(xm,minim)
        plt.xlabel('months')
        plt.title(text_plot)
        
    elif sel == 'Mean':
        meanm=[]
        for month in range(1,13,1):
            Am=var_values[selected_variable][list(np.where(np.asarray(month_list) == month)),:,:]
            meanm=np.append(meanm,Am.mean())
        plt.plot(xm,meanm)
        plt.xlabel('months')
        plt.title(text_plot)
        
    elif sel == 'Deviation':
        dvm=[]
        for month in range(1,13,1):
            Am=var_values[selected_variable][list(np.where(np.asarray(month_list) == month)),:,:]
            dvm=np.append(dvm,Am.std())
        plt.plot(xm,dvm)
        plt.xlabel('months')
        plt.title(text_plot)

In [35]:
print('Choose dataset')
interact_manual(select_dataset, num_dataset=[('Chlorophyll-model', 1), ('Chlorophyll-satellite', 2), ('Oxygen-model', 3), ('Nitrate-model', 4), ('Phosphate-model', 5), ('Suspended matter-satellite', 6)])
print('----------------------------------------------------------------------------------------------')

Choose dataset


interactive(children=(Dropdown(description='num_dataset', options=(('Chlorophyll-model', 1), ('Chlorophyll-sat…

----------------------------------------------------------------------------------------------


In [48]:
print('IMPORTANT: Read the dataset (previous cell) before running this cell for the first time')
print('This interactive map plots the max, min, mean and standard deviation of the variable for each coordinate during a specific year or during the whole timelapse of the dataset')

max_colorbar_input = widgets.BoundedFloatText(value=0,min=0,max=100,step=0.1,description='Max colorbar value:',disabled=False)
min_colorbar_input = widgets.BoundedFloatText(value=0,min=0,max=100,step=0.1,description='Min colorbar value:',disabled=False)
auto_colorbar_checkbox = widgets.Checkbox(value=False,description='Fix manually the values for the colorbar:',disabled=False,indent=False)

interact_manual(plot_map_summary, sel=['Max', 'Min', 'Mean', 'Deviation', 'Argmax', 'Argmin'], year = years_select, fixed_colorbar = auto_colorbar_checkbox, min_colorbar=min_colorbar_input, max_colorbar=max_colorbar_input)

IMPORTANT: Read the dataset (previous cell) before running this cell for the first time
This interactive map plots the max, min, mean and standard deviation of the variable for each coordinate during a specific year or during the whole timelapse of the dataset


interactive(children=(Dropdown(description='sel', options=('Max', 'Min', 'Mean', 'Deviation', 'Argmax', 'Argmi…

<function __main__.plot_map_summary(sel, year, fixed_colorbar, min_colorbar, max_colorbar)>

In [52]:
interact_manual(summary_years, sel=['Max', 'Min', 'Mean', 'Deviation', ], year = years_select)

interactive(children=(Dropdown(description='sel', options=('Max', 'Min', 'Mean', 'Deviation'), value='Max'), B…

<function __main__.summary_years(sel)>

In [18]:
interact_manual(summary_months, sel=['Max', 'Min', 'Mean', 'Deviation', ], year = years_select)

interactive(children=(Dropdown(description='sel', options=('Max', 'Min', 'Mean', 'Deviation'), value='Max'), B…

<function __main__.summary_months(sel)>